<span style="font-size:13px;">

##### ▶ 텍스트 벡터화 : 텍스트 데이터를 메신러닝 모델이 이해할 수 있는 숫자 형태의 벡터로 변환하는 기법
-  BOW (Bag of Words) CountVectorizer
    - 문서를 고정된 길이의 벡터로 변환
    - 문서 - 단어행렬
    - 장점 : 간단하고 빠름,
    - 단점 : 단어순서 손실, 희소성, 의미적 유사성 무시

-  TF-IDF TfidVetorizer
    - 모든 문서에서 자주 등장하는 단어의 영향을 줄이고, 문서의 특이 단어를 강조

-  N-gram 
    - 단어의 순서를 보존하기 위해 N개의 연속된 단어 시퀀스를 하나의 단위로 보는 것
    - 단점 차원폭발에 주의 (정규화/ 차원 축소 고려)

##### ▶ 머신러닝 분류 모델 : 벡터화된 텍스트 데이터를 기반으로 문서를 특정 카테고리로 분류하는 알고리즘
-  multinomal Naive Bayes (다항분포 나이브 베이즈)
    - 확률 모델

-  LogisticRegression 
    - 회귀를 기반으로하지만 시그모이드 함수를 사용하여 클래스를 분류하는데 널리 사용되는 모델
    - 다중클래스 회귀기반 분류

- RidgeClassifer
    - L2규제를 적용한 회귀 기반 분류 모델로 모델의 과적합을 방지


##### ▶ 한국어 형태소 분석기 : 한국어 텍스트를 처리하기 위한 라이브러리
- Kolnpy Okt
---

### <span style="color: gold;">1. 데이터 준비

In [1]:
import numpy as np
import pandas as pd
# scikit-learn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
#분류모델
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# # data load
# newsgroups_train =  fetch_20newsgroups(subset='train'
#                     ,remove = ('headers','footers','quotes')
#                     ,categories=categories
#                    )
# newsgroups_test =  fetch_20newsgroups(subset='test'
#                     ,remove = ('headers','footers','quotes')
#                     ,categories=categories
#                    )

categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

- alt.atheism	무신론, 종교 비판 토론	종교의 존재, 신의 존재 유무, 종교적 주장 반박, 철학적 논쟁 등
- talk.religion.misc	일반 종교 토론 (기타 잡담 포함)	기독교, 불교, 이슬람 등 다양한 종교 관련 이야기, 개인 경험, 신념 공유 등
- comp.graphics	컴퓨터 그래픽스, 이미지 처리	3D 렌더링, 이미지 파일 포맷, 그래픽 소프트웨어 사용법, OpenGL 등 기술 관련 토론
- sci.space 우주 과학, 천문학 로켓, NASA, 행성 탐사, 외계 생명 가능성, 우주 물리학 관련 토론

In [3]:
from sklearn.datasets import load_files
train_path = r'C:\python_src\LLM\20newsbydate\20news-bydate-train'
test_path = r'C:\python_src\LLM\20newsbydate\20news-bydate-test'
newsgroups_train = load_files(train_path,encoding='latin1')
newsgroups_test = load_files(test_path,encoding='latin1')

In [4]:
categories

['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

In [8]:
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [ target_names.index(c) for c in categories  ]
    #필터링
    data_filtered, target_filtered = [], []
    for text,label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)  # 라벨 재 정렬
            data_filtered.append(text) ; target_filtered.append( new_label  )
    return data_filtered,target_filtered,categories

In [9]:
train_data, train_target, target_names = filter_categories(newsgroups_train,categories)
test_data, test_target, _ = filter_categories(newsgroups_test,categories)

In [24]:
np.unique(train_target)

array([0, 1, 2, 3])

---
### <span style="color: gold;"> 2. 텍스트 전처리

In [41]:
# 해더 푸터 인용문 제거

In [10]:
import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.*\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.*$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].*', '', text)

    return text

In [11]:
train_data = [ clean_text(t) for t in train_data]
test_data = [ clean_text(t) for t in test_data]

In [12]:
len(train_data), len(train_target), len(test_data), len(test_target)

(2034, 2034, 1353, 1353)

<span style="font-size:13px;">

#### 멀티노멀 나이즈베이즈  
    - 단어의 **출현 횟수(빈도)**를 세어서, 주어진 문서가 특정 카테고리(예: 스팸, 정상)에 속할 확률을 계산하는 방법
    - 활용분야: 스팸필터링, 뉴스기사 카테고리, 감성분석  
#### 1.  베이즈정리 : 확률 이론 -> 조건부 확률
- 단어 A가 나왔을 때 이 문서가 스팸 B일 확률은 얼마인가  
- $P(\text{스팸} | \text{단어들}) = \frac{P(\text{단어들} | \text{스팸}) \cdot P(\text{스팸})}{P(\text{단어들})}$

#### 2. 나이브 Naive
- 가정: 문서 안의 모든 단어는 서로 독립적이라고 가정
- 현실: 스팸에 자주 나오는 단어들은 서로 독립적이지 않다
- 실제: 이러한 가정은 계산량을 빠르게 하고 단순하지만 정확도가 어느정도 나온다

#### 3. 멀티노멀 : 다항분포
- 의미 : 
    - '다항 분포'를 의미하며, 단어의 출현 횟수를 가장 중요한 정보로 사용
    -  어떤 단어가 단순히 '있다/없다'가 아니라 '몇 번 나왔는가'
- 회수를 세는 멀티노미얼 방식이 NLP와 잘 맞는다
- 모델은 단어의 빈도수 통계
---
- 예시 - 이러한 통계를 바탕으로 이 카테고리에서 특정 단어가 나올 확률 P('free'|스팸)을 모두 계산
-   **스팸 메일 통계**
    -   `free`: 150번
    -   `money`: 100번
    -   `viagra`: 50번
    -   `report`: 5번
-   **정상(Ham) 메일 통계**
    -   `report`: 80번
    -   `meeting`: 60번
    -   `free`: 10번
-   **새로운 메일 분류**  'free money metting'

**1. 이 메일이 '스팸'일 확률 점수 계산**

> **점수(스팸) = P(스팸) × P('free'|스팸) × P('money'|스팸) × P('meeting'|스팸)**  
        -> 기본스팸확률 x 스팸일때 free가 나올 확률 x 스팸일때 money가 나올 확률 x 스팸일 때 meeting 나올 확률                   

-   `P(스팸)`: 전체 메일 중 스팸 메일이 차지하는 기본 확률
-   `P('free'|스팸)`: 스팸 메일에서 'free'가 나올 확률
-   `P('money'|스팸)`: 스팸 메일에서 'money'가 나올 확률
-   `P('meeting'|스팸)`: 스팸 메일에서 'meeting'이 나올 확률

**2. 이 메일이 '정상'일 확률 점수 계산**

> **점수(정상) = P(정상) × P('free'|정상) × P('money'|정상) × P('meeting'|정상)**  
        -> 기본 정상확률 X 정상일때
-   `P(정상)`: 전체 메일 중 정상 메일이 차지하는 기본 확률
-   `P('free'|정상)`: 정상 메일에서 'free'가 나올 확률
-   `P('money'|정상)`: 정상 메일에서 'money'가 나올 확률
-   `P('meeting'|정상)`: 정상 메일에서 'meeting'이 나올 확률

---
### <span style="color: gold;"> 3. 텍스트 벡터화

In [ ]:
# nltk tokenizer stemer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

In [15]:
# min_df : 단어의 빈도가 최소 5개의 문서에 등장  - 노이즈 감소
# max_df : 50% 너무흔한 단어는 제거
cv = CountVectorizer(max_features=2000,min_df=5, max_df=0.5)
x_train_cv = cv.fit_transform(train_data)
x_test_cv = cv.transform(test_data)
x_train_cv.shape,  x_test_cv.shape

((2034, 2000), (1353, 2000))

In [16]:
cv.get_feature_names_out()

array(['00', '000', '01', ..., 'zip', 'zoo', 'zoology'], dtype=object)

In [17]:
(x_train_cv[0].toarray()[0])[x_train_cv[0].toarray()[0]>0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1])

---
### <span style="color: gold;"> 4. 모델 학습 및 평가

<span style="color: pink">1. MultinomialNB (다항분포 나이브 베이즈)</span>

- BoW(CountVectorizer) + MultinomialNB    
- TF-IDF + MultinomialNB    


In [18]:
# BOW 기반 + 멀티노미얼 나이브베이즈 MNB
# 텍스트 분류의 강력한 baseline  희소데이터에 강함
# 모델선택
nb = MultinomialNB()
# 학습용데이터 벡터데이터
nb.fit(x_train_cv,train_target)
print(nb.score(x_train_cv, train_target)  , nb.score(x_test_cv,test_target))
# 분류 리포트
from sklearn.metrics import classification_report
y_pred_nb = nb.predict(x_test_cv)
print( classification_report(test_target,y_pred_nb,target_names=categories)  )

0.9090462143559489 0.804138950480414
                    precision    recall  f1-score   support

       alt.atheism       0.74      0.74      0.74       319
talk.religion.misc       0.67      0.63      0.65       251
     comp.graphics       0.85      0.93      0.89       389
         sci.space       0.89      0.84      0.86       394

          accuracy                           0.80      1353
         macro avg       0.79      0.79      0.79      1353
      weighted avg       0.80      0.80      0.80      1353



In [19]:
# TF-IDF + MNB + LogisticRegression
# TF-IDF로 중요단어 강조, 선형모델과 자주사용  BOW 대비 흔한 단어 영향 감소
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_tfid = tfidf.fit_transform(train_data)
x_test_tfid = tfidf.transform(test_data)

# NB + tf-idf 
nb_tfidf  = MultinomialNB()
nb_tfidf.fit(x_train_tfid,train_target)
print(nb_tfidf.score(x_train_tfid,train_target),  nb_tfidf.score(x_test_tfid, test_target) )

0.9287118977384464 0.8159645232815964


<span style="color: pink">2. LogisticRegression (로지스틱 회귀)</span>
-  TF-IDF + LogisticRegression (기본, L2 규제)
-  TF-IDF + LogisticRegression (L1 규제)

In [20]:
# logistic 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test \
    = train_test_split(x_train_tfid,train_target,test_size=0.2
                       ,stratify=train_target,random_state=42)
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train, y_train)
print(lr.score(x_train,y_train),  lr.score(x_test, y_test) )

0.9704978488014752 0.8894348894348895


<span style="color: pink">3. RidgeClassifier (릿지 분류기)</span>

In [22]:
# 과적합 해결을 위한 규제
rc = RidgeClassifier(alpha=10)
rc.fit(x_train, y_train)
print(rc.score(x_train,y_train),  rc.score(x_test, y_test) )

0.9317762753534112 0.8722358722358723


In [23]:
# L1 규제   L1 Logistic(Lasso와 유사)
# 일부 계수를 0으로 만들어서 특성 선택을 수행.. 중요피처 select 효과
l1_lr = LogisticRegression(penalty='l1',max_iter=1000,solver='saga')
l1_lr.fit(x_train,y_train)
print(l1_lr.score(x_train,y_train),  l1_lr.score(x_test, y_test) )

0.8893669330055316 0.8329238329238329


<span style="color: pink">3. 트리 기반 앙상블 모델</span>
- DecisionTreeClassifier( 결정트리)
- RandomForestClassifier (랜덤 포레스트)
- GradientBoostingClassifier (그래디언트 부스팅)

In [105]:
# 트리모델 + tfidf
tree = DecisionTreeClassifier(max_depth=5)
tree.fit(x_train,y_train)
print(tree.score(x_train,y_train),  tree.score(x_test, y_test) )

fores = RandomForestClassifier(max_depth=10)
fores.fit(x_train,y_train)
print(fores.score(x_train,y_train),  fores.score(x_test, y_test) )

0.510141364474493 0.4963144963144963
0.8543331284572834 0.7616707616707616


In [108]:
gb = GradientBoostingClassifier(max_depth=2)
gb.fit(x_train,y_train)
print(gb.score(x_train,y_train),  gb.score(x_test, y_test) )

0.9446834665027658 0.8329238329238329


In [116]:
x_train,x_test,y_train,y_test = train_test_split(train_data,train_target
         ,stratify=train_target,test_size=0.2,random_state=42)

In [121]:
# 전처리 
# RegexpTokenizer + stopwords + PorterStemmer
english_stops = set(stopwords.words('english'))
regtok = RegexpTokenizer(r"[\w']{3,}")
def custom_tokenizer(text):
    toks = regtok.tokenize(text.lower())
    toks = [t for t in toks if t not in english_stops]
    toks = [PorterStemmer().stem(t) for t in toks]
    return toks
tfidf_custom = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000,min_df=5, max_df=0.5)
x_train_tfidf_c = tfidf_custom.fit_transform(x_train)
x_test_tfidf_c = tfidf_custom.transform(x_test)

c:\Users\playdata2\miniconda3\envs\LLM\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [124]:
from sklearn.linear_model import LogisticRegression
lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_tfidf_c,y_train)
print( lr_c.score(x_train_tfidf_c, y_train) , lr_c.score(x_test_tfidf_c, y_test))

0.9778733866011063 0.8894348894348895


In [133]:
# n-gram 실험 1,2  1,3
# 성능향상 기대  연속된 단어패턴 포착
tfidf_12 = TfidfVectorizer(token_pattern = r"[\w']{3,}"
                           ,stop_words= stopwords.words('english')
                           ,ngram_range=(1,2)
                           , min_df=2, max_df=0.5
                        #    ,max_features=2000
                           )
x_train_12 = tfidf_12.fit_transform(x_train)
x_test_12 = tfidf_12.transform(x_test)

lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_12,y_train)
print( lr_c.score(x_train_12, y_train) , lr_c.score(x_test_12, y_test))

0.9901659496004918 0.9090909090909091


In [1]:
# 한국어 처리  konlpy
# 품사기반 태깅 tokenizer   Noun Verb Adjetive
# 데이터 로딩
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
df.head()

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [2]:
df.title.unique()

array(['인피니티 워', '라라랜드', '곤지암', '신과함께', '범죄도시', '택시운전사', '코코'],
      dtype=object)

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =  train_test_split(df.review
    ,df.title,stratify=df.title, test_size=0.2,random_state=42)

In [6]:
from konlpy.tag import Okt
okt = Okt()

In [19]:
# simple version
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
clf = LogisticRegression(max_iter=1000)

c:\Users\playdata2\miniconda3\envs\LLM\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [22]:
clf.fit(x_train_tfidf,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [23]:
x_train_tfidf.shape  , y_train.shape

((11780, 2000), (11780,))

In [24]:
clf.score(x_train_tfidf,y_train), clf.score(x_test_tfidf,y_test)

(0.7574702886247878, 0.6896434634974533)

In [25]:
def custom_tokenizer(text):
    target = ['Noun','Verb','Adjective']
    return [w for w,tag in okt.pos(text,norm= True, stem=True) if tag in target]

tfidf = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
clf = LogisticRegression(max_iter=1000)
clf.fit(x_train_tfidf,y_train)
clf.score(x_train_tfidf,y_train), clf.score(x_test_tfidf,y_test)

c:\Users\playdata2\miniconda3\envs\LLM\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(0.7824278438030561, 0.7144312393887946)